This notebook cleans the raw housing data acquired from "Heimdal Eiendomsmegling" and merges it with macroeconomic factors.

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder

from Functions.Helpers import classify_broker

pd.set_option('display.max_columns', None)

In [4]:
# Get the dataset and rename the columns.

df = pd.read_excel('../Data/TrondheimHousingData.xlsx')
df = df.rename(columns = {'Adresse': 'Address', 
                          'Boligtype': 'Housing type', 
                          'Prom': 'Primary room', 
                          'BRA': 'Usable area', 
                          'BTA': 'Gross area', 
                          'Byggeår': 'Year of construction', 
                          'Etasje': 'Floor', 
                          'Balkong': 'Balcony', 
                          'Parkering': 'Parking', 
                          'Tomtestørrelse': 'Plot size', 
                          'Sist solgt': 'Last sold', 
                          'Pris': 'Price', 
                          'Fellesgjeld ved siste salg': 'Joint debt at last sale', 
                          'Totalpris ved siste salg': 'Total price at last sale', 
                          'Antall rom': 'Number of rooms', 
                          'Antall soverom': 'Number of bedrooms', 
                          'Hjemmelshaver': 'Owner', 
                          'Registrert dato': 'Registered date', 
                          'Omsetningshastighet (dager)': 'Turnover rate', 
                          'Megler': 'Broker'})

In [4]:
# Remove the Owner column since it does not contain pertinent information.

df.drop(columns = ['Owner'], inplace = True)

In [5]:
# Create a seperate column for the postal code of every residence, might be useful.

df['Postal code'] = df['Address'].str.split(' ').str[-2].astype(int)

Remove all datapoints which meet the following critera:
- Missing dates in the form of "-" in the Last sold column (corresponds to missing dates in the registered date column).
- If the Housing type has been classified to others (because we cannot be sure what kind of residence these could be).
- If the Year of construction is NaN.
- If Number of rooms and Number of bedrooms are both NaN.
- If the Plot size is NaN.

In [6]:
mask = (df['Last sold'] != '-') & (df['Housing type'] != 'Annet') & (df['Year of construction'].notna()) & ((df['Number of rooms'].notna()) | (df['Number of bedrooms'].notna())) & (df['Plot size'].notna())
df = df[mask].reset_index(drop = True)

In [7]:
# Convert the dates to proper date-time format.

df['Registered date'] = pd.to_datetime(df['Registered date'], dayfirst = True)
df['Last sold'] = pd.to_datetime(df['Last sold'], dayfirst = True)

In [8]:
# Fill NaN values with zero and set the type to integer.

df['Floor'] = df['Floor'].fillna(0).astype(int)
df['Primary room'] = df['Primary room'].fillna(0).astype(int)

In [9]:
# Set Usable area to the same value as primary room if missing.

df['Usable area'] = df['Usable area'].fillna(df['Primary room']).astype(int)

# Set Gross area to the same value as Usable area if missing.

df['Gross area'] = df['Gross area'].fillna(df['Usable area']).astype(int)

In [10]:
# Set the number of rooms to the number of bedrooms plus one if missing.

df['Number of rooms'] = df['Number of rooms'].fillna(df['Number of bedrooms'] + 1).astype(int)

# Set the number of bedrooms to the number of rooms minus one if missing.

df['Number of bedrooms'] = df['Number of bedrooms'].fillna(df['Number of rooms'] - 1).astype(int)

In [11]:
# The missing Turnover rate values can be filled with the given dates for Last sold and Registered date.

df['Turnover rate'] = df['Turnover rate'].fillna((df['Last sold'] - df['Registered date']))
df['Turnover rate'] = df['Turnover rate'].apply(lambda x: x.days if isinstance(x, timedelta) else x)
df['Turnover rate'] = df['Turnover rate'].astype(int)

In [12]:
# Calculate the Joint debt at last sale and fill if missing.

joint_debt_at_last_sale = df['Total price at last sale'] - df['Price']
df['Joint debt at last sale'] = df['Joint debt at last sale'].fillna(joint_debt_at_last_sale)

In [13]:
# Classify each broker into their respective mother company, ignore the branches of each company.
# Those companies not large enough, collect into an "Other" category. See Helpers.py.

df['Broker'] = df['Broker'].fillna('Placeholder').apply(classify_broker)

In [14]:
# Encode the brokers and housing types for further use.

encoder = LabelEncoder()

df['Broker encoded'] = encoder.fit_transform(df['Broker'])
df['Housing type encoded'] = encoder.fit_transform(df['Housing type'])

In [15]:
# Create dummy variables for Parking and Balcony Yes/No.

df = pd.get_dummies(df, columns = ['Balcony', 'Parking'], dtype = int)
df.drop(columns = ['Parking_Nei'], inplace = True)
df.rename(columns = {'Balcony_Ja': 'Balcony_Yes', 'Parking_Ja': 'Parking_Yes'}, inplace = True)

In [16]:
# Sort the datapoints in the dataframe by the date each residence was sold.

df = df.sort_values(by = 'Last sold').reset_index(drop = True)

In [17]:
# Add coordinates to the dataset based on the postal code in each datapoint.

dfCoordinates = pd.read_csv('../Data/PostalCodesWithCoordinates.csv')
df = df.merge(dfCoordinates, how = 'left', on = 'Postal code')

In [18]:
# Create seperate columns for year, month, day, of when the residence was sold.

df['Year'] = df['Last sold'].apply(lambda x: x.year)
df['Month'] = df['Last sold'].apply(lambda x: x.month)
df['Day'] = df['Last sold'].apply(lambda x: x.day)

In [7]:
# Import the extra macro data which might be useful.

dfPolicyRate = pd.read_csv('../Data/PolicyRate.csv')
dfMacroData = pd.read_csv('../Data/MacroData.csv')

In [20]:
# Merge the dataset with the macro data.

df = df.merge(dfPolicyRate, how = 'left', on = ['Year', 'Month', 'Day'])
df = df.merge(dfMacroData, how = 'left', on = ['Year', 'Month'])

In [21]:
# Write the cleaned dataframe to an excel file.

df.to_csv('../Data/TrondheimHousingDataCleaned.csv', index = False)

In [22]:
df

,Matrikkel / Org-Anr,Address,Housing type,Primary room,Usable area,Gross area,Year of construction,Floor,Plot size,Last sold,Price,Joint debt at last sale,Total price at last sale,Number of rooms,Number of bedrooms,Registered date,Turnover rate,Broker,Postal code,Broker encoded,Housing type encoded,Balcony_Yes,Parking_Yes,Lat,Lon,Year,Month,Day,Policy rate,CPI,Change CPI,HPI Norway,HPI Trondheim,Borrowing rate%
0,5001-20/88/0/0,"Skogvegen 32, 7058 CHARLOTTENLUND",Selveier enebolig,441,517,567,2007.0,0,970.8,2021-02-28,16900000,0.0,16900000,9,8,2021-02-22,6,Eiendomsmegler 1,7058,3,4,0,0,63.4239,10.4927,2021,2,28,0.0,115.3,0.7,130.7,123.5,1.77
1,5001-410/683/0/34,"Dyre Halses gate 11, 7042 TRONDHEIM",Selveierleilighet,55,55,60,2004.0,7,1043.2,2021-03-01,4380000,0.0,4380000,2,2,2021-02-25,4,DNB Eiendom,7042,1,7,1,0,63.4364,10.4134,2021,3,1,0.0,115.0,-0.3,130.7,123.5,1.77
2,5001-404/418/0/69,"Tellefsens gate 8, 7030 TRONDHEIM",Selveierleilighet,135,144,144,2019.0,5,3673.0,2021-03-01,12500000,0.0,12500000,4,3,2021-01-28,32,Eiendomsmegler 1,7030,3,7,1,0,63.4206,10.3973,2021,3,1,0.0,115.0,-0.3,130.7,123.5,1.77
3,5001-52/224/0/128,"Heggdalsringen 65, 7049 TRONDHEIM",Selveierleilighet,69,73,73,2019.0,5,4572.6,2021-03-01,4950000,0.0,4950000,3,2,2021-03-01,0,Other,7049,8,7,0,0,63.4044,10.4535,2021,3,1,0.0,115.0,-0.3,130.7,123.5,1.77
4,5001-76/267/0/15,"Siriusvegen 10 A, 7037 TRONDHEIM",Selveierleilighet,74,74,81,1995.0,3,9300.8,2021-03-01,3450000,50690.0,3500690,3,2,2021-02-23,6,Eiendomsmegler 1,7037,3,7,1,0,63.3941,10.4113,2021,3,1,0.0,115.0,-0.3,130.7,123.5,1.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20079,5001-408/99/0/8,"Brinken 9, 7016 TRONDHEIM",Selveierleilighet,92,93,93,1912.0,5,538.3,2024-02-28,5300000,17000.0,5317000,3,2,2023-12-24,66,Eiendomsmegler 1,7016,3,7,1,0,63.4256,10.4051,2024,2,28,4.5,134.8,0.0,136.4,131.4,5.56
20080,977248744/16,"Anton Bergs veg 4 A, 7099 FLATÅSEN",Borettslagsleilighet,111,122,130,1998.0,1,8100.0,2024-02-28,3250000,77144.0,3327144,4,2,2024-02-13,15,Heimdal Eiendomsmegling,7099,4,3,0,0,63.3761,10.3505,2024,2,28,4.5,134.8,0.0,136.4,131.4,5.56
20081,5001-95/128/0/0,"Gamle Åsvei 61, 7020 TRONDHEIM",Selveier enebolig,175,242,0,1925.0,0,801.0,2024-02-28,7750000,0.0,7750000,4,3,2024-02-01,27,DNB Eiendom,7020,1,4,1,0,63.4194,10.3541,2024,2,28,4.5,134.8,0.0,136.4,131.4,5.56
20082,5001-411/201/0/48,"Anders Buens gate 19, 7067 TRONDHEIM",Selveierleilighet,46,54,54,1939.0,4,1533.2,2024-02-28,2780000,130300.0,2910300,2,1,2024-02-05,23,Eiendomsmegler 1,7067,3,7,1,0,63.4380,10.4277,2024,2,28,4.5,134.8,0.0,136.4,131.4,5.56
